In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Define elements to use:

used_els = ['Fe', 'He', 'Mg', 'Ne', 'O', 'Si'] # TNG elements but removing C and N for proto-solar corrections

In [5]:
# Load in full training data
full_input = '/home/oliver/ChempyMulti/ChempyMultiData/All_Timestep_Training_Predictions.npz'
dat=np.load(full_input,mmap_mode='r')
all_els = dat['elements']
params=dat['params']
times=dat['timestep']

In [6]:
abun=dat.f.abundances

In [7]:
el_indices=np.zeros(len(used_els),dtype=int)
for e,el in enumerate(used_els):
    el_indices[e]=np.where(el==all_els)[0]

In [8]:
# Filter out unwanted elements
new_arr = abun[:,el_indices]

In [9]:
full_params=np.zeros([len(params)*len(times),6])
for i in range(len(params)*len(times)):
    full_params[i,:5]=params[i//len(times)]
    full_params[i,5]=times[i%len(times)]

ValueError: could not broadcast input array from shape (6) into shape (5)

In [215]:
full_norm_params=np.zeros([len(params)*len(times),6])
for i in range(len(norm_params)*len(times)):
    full_norm_params[i,:5]=norm_params[i//len(times)]
    full_norm_params[i,5]=times[i%len(times)]

Save array

In [256]:
dat.close()

np.savez("/mnt/store1/oliverphilcox/TrainingPredictions6El.npz",abundances=new_arr,elements=used_els,params=full_params,norm_params=full_norm_params)

In [257]:
# Test input
dat2=np.load('/mnt/store1/oliverphilcox/TrainingPredictions6El.npz')

In [258]:
ab=dat2.f.abundances

In [259]:
bmap=np.zeros(len(ab),dtype=int)
for i,a in enumerate(ab):
    if a[0]!=0:
        bmap[i]=1

In [278]:
good_par=par[np.where(bmap==0)]
ints=np.random.randint(0,len(good_par),1000)

We note that $\sim7\%$ of the data is dodgy here, which results in empty arrays, indicating that the model could not be run for this set of parameters.

In [293]:
sum(bmap)

19136250

Create new training data parameter set:

In [343]:
import os
os.chdir('../')
from Chempy.parameter import ModelParameters
a=ModelParameters()
import multiprocessing as mp
import numpy as np
import tqdm
import time
from Chempy.cem_function import multi_timestep_chempy


# First create training dataset
# Define training size
N = 8  # 8 grid points per parameter
widths = np.asarray([0.6, 0.6, 0.6, 0.2, 0.2]) # true training data distribution widths

from scipy.stats import norm as gaussian
prob = np.linspace(1./(N+1.), 1.-1/(N+1), int(N))
grids = [gaussian.ppf(prob) for _ in range(len(a.p0))] # Normalize to unit Gaussian
norm_grid = np.array(np.meshgrid(*grids)).T.reshape(-1,len(a.p0))

times = np.linspace(1,13.5,N)

# Create grid in parameter space
full_widths = list(widths)
means = list(a.p0)
param_grid = [np.asarray(item)*full_widths+means for item in norm_grid]


all_params=[]
for par in param_grid:
    for t in times:
        all_params.append(np.asarray(list(par)+[t]))

In [362]:
10**6

1000000